In [6]:
# source nike_env/bin/activate to get into virtual environment
# pip3 install -r requirements.txt (Install requirements file and packages)
#add all install packages to requirements.txt
# run pip3 freeze > requirements.txt to get any imports into requirements.txt
import pandas as pd
import requests
from datetime import datetime
import matplotlib.pyplot as plt
from datetime import datetime, timedelta


In [9]:
# libraries for twitter sentiment analysis 
import tweepy
import pandas as pd
import random
import numpy as np
import pandas as pd
from pathlib import Path
import clean
import warnings

In [ ]:
consumer_key    = 'PjwX6vanPHpnxoUtbSgMzRgRi'
consumer_secret = '73JxkiEI9bK3OmqRncjwwhuUVdY50bUG3WBAXIBZyqg91PMYbE'

access_token  = '1533324119080546304-T0gud2D5jqUfiTRxDODoomj8YtAwJN'
access_token_secret = 'dedGpyOQtUNMRo1z04XQ9UPNErz8OMJUDYQdPeW74xhju'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
pd.set_option('display.max_colwidth', -1)


warnings.filterwarnings('ignore')
np.random.seed(37)


nikeTweetPath = Path(__file__).absolute().parent.joinpath('../dataset/5000-justdoit-tweets-dataset/justdoit_tweets_2018_09_07_2.csv')
tweet_data = pd.read_csv(nikeTweetPath)


tweet_data = tweet_data[['tweet_created_at','tweet_full_text']]
tweet_data = tweet_data.dropna()
tweet_data = tweet_data.reindex(np.random.permutation(tweet_data.index))
#training_data['sentiment'] = training_data['sentiment'].map({'negative': -1, 'positive': 1,'neutral':.5})
ct = clean.CleanText()
sr_clean = ct.fit_transform(tweet_data.tweet_full_text)
tweet_data['text_clean'] = sr_clean

tweet_data.to_csv(Path(__file__).absolute().parent.joinpath('../dataset/nike-preprocessed.csv'))


fetch_tweets=tweepy.Cursor(api.search, q="#NIKE",count=100, lang ="en",since="2018-9-13", tweet_mode="extended").items()
data=pd.DataFrame(data=[[tweet_info.created_at.date(),tweet_info.full_text]for tweet_info in fetch_tweets],columns=['Date','Tweets'])

In [4]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=NKE&interval=60min&apikey=CZS7XN9EGAHLU74N&outputsize=full'
r = requests.get(url)
print(f"Status code: {r.status_code}")

Status code: 200


In [6]:
nike_stock_over_time = r.json()

{'Meta Data': {'1. Information': 'Daily Time Series with Splits and Dividend Events',
  '2. Symbol': 'NKE',
  '3. Last Refreshed': '2023-05-02',
  '4. Output Size': 'Full size',
  '5. Time Zone': 'US/Eastern'},
 'Time Series (Daily)': {'2023-05-02': {'1. open': '127.15',
   '2. high': '127.58',
   '3. low': '125.165',
   '4. close': '127.26',
   '5. adjusted close': '127.26',
   '6. volume': '3831644',
   '7. dividend amount': '0.0000',
   '8. split coefficient': '1.0'},
  '2023-05-01': {'1. open': '126.92',
   '2. high': '128.68',
   '3. low': '126.86',
   '4. close': '127.92',
   '5. adjusted close': '127.92',
   '6. volume': '4112782',
   '7. dividend amount': '0.0000',
   '8. split coefficient': '1.0'},
  '2023-04-28': {'1. open': '125.15',
   '2. high': '127.07',
   '3. low': '124.96',
   '4. close': '126.72',
   '5. adjusted close': '126.72',
   '6. volume': '4421419',
   '7. dividend amount': '0.0000',
   '8. split coefficient': '1.0'},
  '2023-04-27': {'1. open': '124.44',
   '

In [ ]:
# Cleaning the text

import re
import nltk

def get_clean_text(title, folder):
    """Given a character name (title) and a folde in which to find it
    return the page markup as a neatly cleaned string.
    
    Input
    -----
        title : str
        folder : str
    
    Output
    ------
        text : str
        
    Example
    -------
        >>> get_clean_text('Iron Man', 'heroes')
    """
    # Load markup
    with open(f"../data/{folder}/{title}.txt") as fp:
        text = fp.read()

    # Remove category links
    text = re.sub(r'\[\[Category.+\]\]', '', text)
    
    # Fix links (match and clear "[[Iron Man (comic book)|" and "[[", then on next line "]]"
    text = re.sub(r'(\[\[((?!\]\]).)+\|)|(\[\[)', '', text)
    text = re.sub(r'\]\]', '', text)
    
    # Remove '''
    text = re.sub(r"'''", '', text)
    
    # Remove refs
    text = re.sub(r'<ref.+?</ref>', '', text)
    
    # Remove other ugly html links
    text = re.sub(r'<.+?>', '', text)
    
    # Remove '=' signs in headers
    text = re.sub(r'=+', '', text)
    
    # Remove table and external links
    text = re.sub(r'\{\{[\w\W]*?\}\}', '', text)
    
    # Remove everything after "See Also"
    text = re.sub(r'== ?See [aA]lso[\w\W]+', "", text)

    return text

In [ ]:
# imports

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 300)
import praw #reddit data api
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn
import re #regex
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer #VADER sentiment model
import requests
import json
import csv
import time
import datetime
import praw

In [ ]:
# Subreddit to query
sub='stocks'
# Start and end dates
before = "1594339200" #july 10 2020
after = "1498867200" #july 1 2017
# Query string (i.e. thread we'd like to parse)
query = "Daily Discussion"
subCount = 0
subStats = []

In [ ]:
# pulls subreddit data from specific subreddit between specified dates
def getPushshiftData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

# parse the retrieved data to make it interprettable for sentiment analysis
def collectSubData(subm):
    subData = [subm['id'], subm['title'], subm['url'], datetime.datetime.fromtimestamp(subm['created_utc']).date()]
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"
    subData.append(flair)
    subStats.append(subData)


data = getPushshiftData(query, after, before, sub)
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftData(query, after, before, sub)

In [ ]:
# Create dataframe to hold data
data={}
ids=[]
titles=[]
urls=[]
dates=[]
flairs=[]
for stat in subStats:
    ids.append(stat[0])
    titles.append(stat[1])
    urls.append(stat[2])
    dates.append(stat[3])
    flairs.append(stat[4])
data['id']=ids
data['title']=titles
data['url']=urls
data['date']=dates
data['flair']=flairs
df_1=pd.DataFrame(data)
df_1=df_1[df_1['flair']=='Daily Discussion']

In [ ]:
# Connect to Reddit API using our secret key and web app credentials
reddit = praw.Reddit(client_id='V00kIJZzUbbg4J89HASw5w', client_secret='kdV12zDMY1cBmpBEMyQi9fhKVTwN0g', user_agent='ai prac')

# Collect and query comments using PRAW
comments_by_day=[]
for url in df_1['url'].tolist():
    try:
        submission = reddit.submission(url=url)
        submission.comments.replace_more(limit=0)
        comments=list([(comment.body) for comment in submission.comments])
    except:
        comments=None
    comments_by_day.append(comments)

In [ ]:
analyser = SentimentIntensityAnalyzer()

scores=[]
for comments in comments_by_day:
    sentiment_score=0
    try:
        for comment in comments:
            sentiment_score=sentiment_score+analyser.polarity_scores(comment)['compound']
    except TypeError:
        sentiment_score=0
    
    scores.append(sentiment_score)
    
df_1['sentiment score']=scores

In [ ]:
# Combine with S&P 500 price and plot
"""
spy_url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=SPY&interval=60min&apikey=CZS7XN9EGAHLU74N&outputsize=full'
spy_r = requests.get(url)
print(f"Status code: {spy_r.status_code}")
spy_over_time = r.json()
# spy = ffn.get('spy', start='2010-01-01')
spy_vals=[]
for date in df_1['date'].tolist():
    try:
        spy_vals.append(float(spy.loc[date]))
    except KeyError:
        spy_vals.append(None)
        
df_1['spy']=spy_vals """

spy=ffn.get('spy', start='2010-01-01')
spy_vals=[]
for date in df_1['date'].tolist():
    try:
        spy_vals.append(float(spy.loc[date]))
    except KeyError:
        spy_vals.append(None)
        
df_1['spy']=spy_vals

In [ ]:
df_1=df_1[['date','sentiment score','spy']]
df_1=df_1.set_index('date')
df_1=df_1[df_1['spy'].notna()]

df_1.to_csv('positive+negative_sentiment_data_1.csv')

df_1.plot(secondary_y='sentiment score', figsize=(16, 10))

In [ ]:
# Fast Fourier Transform is used to discretize and smoothen price curve

close_fft = np.fft.fft(np.asarray(df_1['sentiment score'].tolist()))
fft_df = pd.DataFrame({'fft':close_fft})
fft_df['absolute'] = fft_df['fft'].apply(lambda x: np.abs(x))
fft_df['angle'] = fft_df['fft'].apply(lambda x: np.angle(x))
fft_list = np.asarray(fft_df['fft'].tolist())

for num_ in [5, 10, 15, 20]:
    fft_list_m10= np.copy(fft_list); fft_list_m10[num_:-num_]=0
    df_1['fourier '+str(num_)]=np.fft.ifft(fft_list_m10)

In [ ]:
df_1[['sentiment score', 'fourier 5', 'fourier 10', 'fourier 15', 'fourier 20']].plot(figsize=(16, 10));

In [ ]:
df_1[['spy', 'fourier 20']].plot(secondary_y='fourier 20', figsize=(16, 10));

In [ ]:
# Normalize using MinMaxScaling
from sklearn.preprocessing import MinMaxScaler
sc= MinMaxScaler(feature_range=(0,1))
df_1['norm_price']=sc.fit_transform(df_1['spy'].to_numpy().reshape(-1, 1))
df_1['spy log']=np.log(df_1['spy']/df_1['spy'].shift(1))
df_1['norm_sentiment']=sc.fit_transform(df_1['sentiment score'].to_numpy().reshape(-1, 1))
df_1['norm_fourier5']=sc.fit_transform(np.asarray(list([(float(x)) for x in df_1['fourier 5'].to_numpy()])).reshape(-1, 1))
df_1['norm_fourier10']=sc.fit_transform(np.asarray(list([(float(x)) for x in df_1['fourier 10'].to_numpy()])).reshape(-1, 1))
df_1['norm_fourier15']=sc.fit_transform(np.asarray(list([(float(x)) for x in df_1['fourier 15'].to_numpy()])).reshape(-1, 1))
df_1['norm_fourier20']=sc.fit_transform(np.asarray(list([(float(x)) for x in df_1['fourier 20'].to_numpy()])).reshape(-1, 1))

In [ ]:
df_1[['norm_price', 'norm_sentiment', 'norm_fourier5', 'norm_fourier20']].plot(figsize=(16, 10));

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()
category_scores = {"nike": [[], [], []], "competitors": [[], [], []], "general_economy": [[], [], []],}